In [32]:
import pandas as pd
import yaml
import time
import os
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import logging

In [33]:
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for more verbosity
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [99]:
def scrape_edsresults(url, year, race_date, race_name, race_loc, race_dist, race_id):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    rows = soup.find_all('tr')  # Try table rows instead

    logging.info(f"Scraping: {url}")
    logging.info(f"Total rows found: {len(rows)}")

    data = []
    for row in rows:
        cells = row.find_all('td')
        num_cells = len(cells)
        
        if num_cells < 14:
            logging.debug("Skipping row with insufficient cells")
            continue
        
        # Safely extract finish_time and status
        try:
            if num_cells >= 16:
                finish_time = cells[14].get_text(strip=True)
                raw_status = cells[15].get_text(strip=True)
            elif num_cells >= 15:
                finish_time = cells[13].get_text(strip=True)
                raw_status = cells[14].get_text(strip=True)
            else:
                finish_time = cells[13].get_text(strip=True)
                raw_status = cells[14].get_text(strip=True)
        except IndexError:
            logging.warning(f"Row with {num_cells} cells caused IndexError. Skipping.")
            continue
    
        # Normalize and validate status
        raw_status = raw_status.strip().upper()
        if raw_status == 'COMPLETE':
            status = 'Finisher'
        elif raw_status == 'DNF':
            status = 'DNF'
        elif raw_status == 'DNS':
            logging.debug(f"Skipping DNS entry: {cells[4].get_text(strip=True)} {cells[5].get_text(strip=True)}")
            continue
        else:
            logging.debug(f"Unexpected status value '{raw_status}' for row: {cells}")
            continue


        first_name = cells[4].get_text(strip=True)
        last_name = cells[5].get_text(strip=True)
        name = f"{first_name} {last_name}"

        raw_status = cells[14].get_text(strip=True)
        finish_time = cells[13].get_text(strip=True)

        logging.info(f"Year {year} has {num_cells} columns per row")

        data.append({
            'Date': race_date,
            'Year': year,
            'Rank': cells[0].get_text(strip=True),
            'First_Name': first_name,
            'Last_Name': last_name,
            'Name': name,
            'City': cells[8].get_text(strip=True),
            'State': cells[9].get_text(strip=True),
            'Age': cells[7].get_text(strip=True),
            'Gender_Div': cells[6].get_text(strip=True),
            'Div_Place': cells[2].get_text(strip=True),
            'Time': finish_time,
            'Status': status
        })

    logging.info(f"Finished scraping {len(data)} results for {race_name} {year}")
    return pd.DataFrame(data)

In [100]:
def load_race_config(race_file):
    folder = "edsresults_yaml"
    path = os.path.join("..", "config", folder, race_file)

    with open(path, "r") as f:
        return yaml.safe_load(f)

In [101]:
race = load_race_config("bandera_100k.yaml")

In [102]:
all_results = []

for year, slug in race["race_slugs"].items():
    logging.info(f"Processing year {year} with slug {slug}")
    url = f"http://edsresults.com/{slug}/index.php?search_type=race_results&event=100K&gender=&results_per_page=400"
    race_date = race["race_date"][int(year)]
    df = scrape_edsresults(
    url,
    year,
    race_date,
    race["race_name"],
    race["race_loc"],
    race["race_dist"],
    race.get("race_id", None)
)


    df['Race'] = race["race_name"]
    df['Race_Loc'] = race["race_loc"]
    df['Race_Dist'] = race["race_dist"]
    df['Year'] = int(year)
    df['Race_ID'] = race["race_id"]
    all_results.append(df)

race_df = pd.concat(all_results, ignore_index=True)

2025-11-02 13:15:16,495 - INFO - Processing year 2022 with slug bandera22
2025-11-02 13:15:25,542 - INFO - Scraping: http://edsresults.com/bandera22/index.php?search_type=race_results&event=100K&gender=&results_per_page=400
2025-11-02 13:15:25,543 - INFO - Total rows found: 404
2025-11-02 13:15:25,544 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,544 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,545 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,545 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,545 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,546 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,548 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,549 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,551 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,552 - INFO - Year 2022 has 15 columns per row
2025-11-02 13:15:25,553 - INFO - Year 2022 has 15 columns per

In [103]:
race_df

,Date,Year,Rank,First_Name,Last_Name,Name,City,State,Age,Gender_Div,Div_Place,Time,Status,Race,Race_Loc,Race_Dist,Race_ID
0,"January 8, 2022",2022,1,Jonathan,Rea,Jonathan Rea,Boulder,CO,29,M,1,08:10:00,Finisher,Bandera 100k,"Bandera, TX",100K,50001
1,"January 8, 2022",2022,2,Tyler,Fox,Tyler Fox,Lander,WY,29,M,2,08:14:53,Finisher,Bandera 100k,"Bandera, TX",100K,50001
2,"January 8, 2022",2022,3,Joseph,McConaughy,Joseph McConaughy,Seattle,WA,30,M,3,08:17:34,Finisher,Bandera 100k,"Bandera, TX",100K,50001
3,"January 8, 2022",2022,4,Rod,Farvard,Rod Farvard,Mammoth Lakes,CA,26,M,4,08:28:39,Finisher,Bandera 100k,"Bandera, TX",100K,50001
4,"January 8, 2022",2022,5,Jimmy,Elam,Jimmy Elam,Salt Lake City,UT,34,M,5,08:33:07,Finisher,Bandera 100k,"Bandera, TX",100K,50001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527,"January 11, 2025",2025,,Ben,Gale,Ben Gale,Markleville,IN,46,M,,05:11:16,DNF,Bandera 100k,"Bandera, TX",100K,50001
1528,"January 11, 2025",2025,,Scott,Custead,Scott Custead,Burleson,TX,46,M,,N/A,DNF,Bandera 100k,"Bandera, TX",100K,50001
1529,"January 11, 2025",2025,,andre,hermann,andre hermann,Sugar Land,TX,50,M,,N/A,DNF,Bandera 100k,"Bandera, TX",100K,50001
1530,"January 11, 2025",2025,,Martin,Ortiz,Martin Ortiz,Austin,TX,54,M,,N/A,DNF,Bandera 100k,"Bandera, TX",100K,50001


In [104]:
status_counts = race_df.groupby(['Year', 'Status']).size().unstack(fill_value=0)
status_counts

Status,DNF,Finisher
Year,,
2022,78,311
2023,80,320
2024,65,324
2025,122,232


In [109]:
race_df['Year'].value_counts()

Year
2023    400
2022    389
2024    389
2025    354
Name: count, dtype: int64

In [110]:
bandera_100k_df = race_df

In [111]:
bandera_100k_df.to_csv('../data/ultra_raw_csv/100 K/bandera_100k_df_raw.csv', index = False, encoding = 'utf-8')